# Assignment 3 Primer

* Assignment 3 will be based on a sample of movie rating predictions from the Netflix Prize dataset.
* It includes two pregenerated splits of 10k users for about 3k movies. So there is already a train and validation split that you will use for your experiments.
* It can be a little tricky to use predefined splits in surpise, so I will show you how to do it here.
* There are many articles available that talk about the million dollar prize and what it meant for the company. A few are:
* https://towardsdatascience.com/deep-dive-into-netflixs-recommender-system-341806ae3b48
* https://www.thrillist.com/entertainment/nation/the-netflix-prize
* https://analyticsindiamag.com/how-useful-was-the-netflix-prize-really/

In [1]:
import numpy as np
import pyarrow.feather as feather
import pandas as pd

# If you are on an Intel-based machine, you can also enable scikit-learn intel optimizations.
# This does not work on an M1 / M2 Macbook, but does on older Macbooks with an Intel.
# Requires ``pip install scikit-learn-intelex''
#
# from sklearnex import patch_sklearn
# patch_sklearn()

In [2]:
import pyarrow.feather as feather
import pandas as pd
from surprise import Dataset, Reader, BaselineOnly, accuracy
from surprise.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from joblib import parallel_backend
import os, psutil



In [3]:
df_train = feather.read_feather('netflix-5k.train.feather')
df_val = feather.read_feather('netflix-5k.validation.feather')
df_titles = feather.read_feather('netflix-5k.movie_titles.feather')

In [4]:
display(df_titles)

,movieID,imdb_id,imdb_date,date,title,genre
0,3,1860,1997,1997,Character,Drama
1,8,27899,2004,2004,What the #$*! Do We Know!?,Comedy|Documentary|Drama
2,16,76,1995,1996,Screamers,Action|Sci-Fi|Thriller
3,18,249,1994,1994,Immortal Beloved,Drama|Romance
4,26,7368,2004,2004,Never Die Alone,Crime|Drama|Thriller
...,...,...,...,...,...,...
4329,17761,6285,2003,2003,Levity,Drama
4330,17762,1653,1997,1997,Gattaca,Drama|Sci-Fi|Thriller
4331,17763,3813,1978,1978,Interiors,Drama
4332,17764,2396,1998,1998,Shakespeare in Love,Comedy|Drama|Romance


In [5]:
display(df_train)

,userID,movieID,rating
0,1000596,10036,4.0
1,1000596,10100,2.0
2,1000596,10209,3.0
3,1000596,10212,3.0
4,1000596,10225,3.0
...,...,...,...
1287012,99993,9683,4.0
1287013,99993,9685,4.0
1287014,99993,9800,2.0
1287015,99993,9939,5.0


In [6]:
display(df_val)

,userID,movieID,rating
0,1000596,10827,4.0
1,1000596,11089,4.0
2,1000596,11279,3.0
3,1000596,11337,3.0
4,1000596,1180,3.0
...,...,...,...
321485,99993,8764,5.0
321486,99993,9092,4.0
321487,99993,9508,4.0
321488,99993,9788,3.0


In [7]:
df_val.sample(10, random_state=999)

,userID,movieID,rating
116524,1862022,4914,3.0
192023,2417987,14670,3.0
159282,2185341,8512,3.0
173060,2279915,12365,4.0
103558,1767267,11678,2.0
43284,1313635,4109,2.0
153839,2137416,4043,3.0
314713,949177,9687,2.0
34067,1251010,17088,4.0
147533,2087843,17346,4.0


In [8]:
df_train.sample(10, random_state=999)

,userID,movieID,rating
120255,1219163,9340,3.0
261051,1474206,16954,5.0
142843,1261019,4745,4.0
153254,1278881,15764,3.0
371995,1681954,7401,3.0
739346,2369506,10906,5.0
786562,2450375,7904,2.0
377812,1692112,6974,5.0
735414,2362939,7234,2.0
91197,1163417,13923,4.0


In [9]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df_train[['userID','movieID',
                                      'rating']],reader)
datav = Dataset.load_from_df(df_val[['userID','movieID',
                                       'rating']],reader)
trainset = data.build_full_trainset()
NA,valset = train_test_split(datav, test_size=1.0)

print ('Run simple baseline model')
bl = BaselineOnly()
model = bl.fit(trainset)
predictions = bl.test(valset)
print('RMSE = {}'.format(round(accuracy.rmse(predictions,verbose=False),3)))

Run simple baseline model
Estimating biases using als...
RMSE = 0.9


In [10]:
display(predictions)

[Prediction(uid=2104114, iid=6450, r_ui=4.0, est=3.9362292386374755, details={'was_impossible': False}),
 Prediction(uid=1240814, iid=10230, r_ui=5.0, est=3.373959357091867, details={'was_impossible': False}),
 Prediction(uid=1380094, iid=7166, r_ui=5.0, est=4.78256791940289, details={'was_impossible': False}),
 Prediction(uid=1296115, iid=16879, r_ui=5.0, est=4.038569337377335, details={'was_impossible': False}),
 Prediction(uid=2281128, iid=313, r_ui=4.0, est=3.612697189459912, details={'was_impossible': False}),
 Prediction(uid=873352, iid=4570, r_ui=4.0, est=3.5955328150419046, details={'was_impossible': False}),
 Prediction(uid=150663, iid=17136, r_ui=2.0, est=3.512323333507219, details={'was_impossible': False}),
 Prediction(uid=1770703, iid=2112, r_ui=5.0, est=4.23285119763595, details={'was_impossible': False}),
 Prediction(uid=249413, iid=4364, r_ui=3.0, est=3.7087939513917205, details={'was_impossible': False}),
 Prediction(uid=1182394, iid=6302, r_ui=4.0, est=3.1915435824800

In [11]:
# Let's build a pandas dataframe with all the predictions

def get_Iu(uid):
    """Return the number of items rated by given user
    
    Args:
        uid: The raw id of the user.
    Returns:
        The number of items rated by the user.
    """
    
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError:  # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """Return the number of users that have rated given item
    
    Args:
        iid: The raw id of the item.
    Returns:
        The number of users that have rated the item.
    """
    
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:  # item was not part of the trainset
        return 0

df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])    
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
df = df.drop(['details'],axis=1)

In [10]:
display(df)

,uid,iid,rui,est,Iu,Ui,err
0,341042,8393,4.0,4.163322,223,1123,0.163322
1,1922931,16016,1.0,3.204906,137,545,2.204906
2,1337502,4306,5.0,4.699602,359,2272,0.300398
3,1603456,4977,3.0,3.221169,246,613,0.221169
4,283345,10157,4.0,3.751336,298,29,0.248664
...,...,...,...,...,...,...,...
321485,1304363,548,1.0,2.975195,209,130,1.975195
321486,251789,10607,5.0,4.163938,277,956,0.836062
321487,1281631,15058,5.0,4.075456,270,1111,0.924544
321488,2618067,10168,3.0,2.452232,367,1545,0.547768


## Now let's look at the API documents for the 'baseline' only model in surprise: https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#baseline-estimates-configuration

## Or after some digging, even more informative -- https://github.com/NicolasHug/Surprise/blob/master/doc/source/prediction_algorithms.rst

In [11]:
# So let's setup a grid search based on these options. For simplicity, we will look at ALS only.

param_grid = {'bsl_options': 
                  {'method':['als'],'n_epochs':[5, 10, 20, 30, 50], 
                   'reg_i': [8, 9, 10, 11, 12], # lambda 2
                   'reg_u': [3, 4, 5, 6, 7],# lambda 3
                  }
             }

unwrapped_grid = {'method':['als'],'n_epochs':[5, 10, 20, 30, 50], 
                   'reg_i': [8, 9, 10, 11, 12], # lambda 2
                   'reg_u': [3, 4, 5, 6, 7],# lambda 3
                  }

# Seems OK. How permutations are there exactly?
from sklearn.model_selection import ParameterGrid
pg = ParameterGrid(unwrapped_grid)
print ('Total Permutations = ',len(pg))

Total Permutations =  125


In [12]:
# Now let's run it.
# I am forcing joblib to use multiprocessing to avoid problems with failed threads.
# Be conservative with the n_jobs you choose if you have limited memory. You can set
# it to -1 if you have a lot, which will use all of the cores. If it crashes, try using
# only one. Make sure you set n_jobs in both calls to the same value.

with parallel_backend('multiprocessing', n_jobs=1):
    gs = GridSearchCV(BaselineOnly, param_grid, joblib_verbose=10, cv=5,
                          measures=['rmse'], n_jobs=1)
    gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

rmse_params = gs.best_params['rmse']

bl_tuned = BaselineOnly(rmse_params)
model = bl_tuned.fit(trainset)
predictions = bl_tuned.test(valset)
print('RMSE = {}'.format(round(accuracy.rmse(predictions,verbose=False),3)))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Estimating biases using als...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


Estimating biases using als...


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.7s remaining:    0.0s


...

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimati

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimati

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimati

[Parallel(n_jobs=1)]: Done 625 out of 625 | elapsed: 30.9min finished


0.9020584625005847
{'bsl_options': {'method': 'als', 'n_epochs': 50, 'reg_i': 8, 'reg_u': 3}}
Estimating biases using als...
RMSE = 0.9


In [13]:
# That was really slow, so let's setup a randomized search based on these options. 
# For simplicity, we will look at ALS only.

param_grid = {'bsl_options': 
                  {'method':['als'],'n_epochs':[5, 10, 20, 30, 50], 
                   'reg_i': [8, 9, 10, 11, 12], # lambda 2
                   'reg_u': [3, 4, 5, 6, 7],# lambda 3
                  }
             }
with parallel_backend('multiprocessing', n_jobs=1):
    gs = RandomizedSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=5, n_iter=8, 
                            n_jobs=1, joblib_verbose=10)
    gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

rmse_params = gs.best_params['rmse']

bl_tuned = BaselineOnly(rmse_params)
model = bl_tuned.fit(trainset)
predictions = model.test(valset)
print('RMSE = {}'.format(round(accuracy.rmse(predictions,verbose=False),3)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Estimating biases using als...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s


Estimating biases using als...


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.1s remaining:    0.0s


...

Estimating biases using als...


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   25.3s remaining:    0.0s


Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  2.2min finished


0.9021646969160105
{'bsl_options': {'method': 'als', 'n_epochs': 50, 'reg_i': 8, 'reg_u': 7}}
Estimating biases using als...
RMSE = 0.9


## Note that while my attempt to tune the BaselineOnly method did not result in any noticeable performance improvements based on RMSE, that is not entirely surprising based on how this algorithm works. My goal was to show you how to tune the parameters using two different methods, so that you could more easily test and tune other algorithms from surprise.

In [14]:
# Print the memory usage of this process. Requires you to set n_jobs=1 and ``pip install psutil''
# Use this to debug any memory issues you might be having.
# process = psutil.Process(os.getpid())
# print(process.memory_info().rss/(1024*1024))  # in MB

1139.046875
